# netCDF4读取
1. f.variables 得到变量列表，其中key为变量名，values为nc格式的数据
2. 时间格式转换：nc.num2date()/datetime/pandas
3. 属性常用关键字，在读取文件时会自动计算

In [1]:
import netCDF4 as nc
import numpy as np

## f.variables

In [ ]:
inpath = (fr"E:/ERA5/adaptor.mars.internal-1706248324.889576-22324-13-f47c138e-3d82-4878-813e-99a8405dd3d5.nc")
with nc.Dataset(inpath, mode='r') as f:
    var_dict = f.variables
    var_keys = var_dict.keys()
    for key, var in var_dict.items():
        data = var[:] 
    
    for attr_name in var.ncattrs():
        attr_value = getattr(var, attr_name)
        print(attr_name, attr_value)


########### 常用复制-粘贴
# print(f.variables.keys())
# var_keys = list(var_dict.keys())
# print(f.variables[var_name], "\n")
# lat = (f.variables['latitude'][:])
# lon = (f.variables['longitude'][:])
# level = (f.variables['level'][:])
# time = (f.variables['time'][:])

## 时间格式转换
> 其中datetime仅可进行单个元素的转换，支持float类型  
> float 和 array of float的区别是：float32 视为单个浮点数的数据类型，而 array of float32 是一种容器，用于存储多个 float32 值的数据结构。  
举个例子，假设你有一个名为 temperature 的变量，它存储一个月份内每天的温度值。如果你使用 float32 数据类型，你只能存储一个温度值，而如果你使用 array of float32 数据结构，你可以存储一个包含一个月份内所有温度值的数组。

In [2]:
import cftime
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
### cftime
t = nc.num2date(time, 'hours since 1900-01-01 00:00:0.0').data

### datetime
# 定义参考时间（1900-01-01 00:00:00.0）
reference_time = datetime(1900, 1, 1, 0, 0, 0)
# 计算时间偏移量
time_offset = timedelta(hours=float(time[0]))
# 计算转换后的日期时间
converted_time = reference_time + time_offset

### pandas
time_series = pd.to_datetime(time, unit='h', origin='1900-01-01')

## 常用属性关键字
> **scale_factor 和 add_offset** 属性的目的是在存储和传输数据时进行数据压缩和优化，在读取数据时会进行自动校正！  
**scale_factor** 允许将原始数据乘以一个因子，从而将数据的范围缩放到更小的范围内。这样可以降低数据的存储需求和传输带宽。例如，如果原始数据表示温度，范围是从 -100 到 100，但你可以使用 scale_factor 将数据缩放到 -1 到 1 的范围内，以减小存储和传输的数据量。  
**add_offset** 允许将一个固定值添加到原始数据上，从而进行偏移。这对于调整数据的零点或基准值非常有用。例如，如果原始数据表示海拔高度，以米为单位，但你可以使用 add_offset 将数据的零点偏移到海平面上，以便存储和传输相对海平面的高度。

1. scale_factor：缩放因子，用于将数据乘以一个固定值进行缩放。
2. add_offset：偏移量，用于将一个固定值添加到数据上进行偏移。
3. units：变量的单位。
4. long_name：变量的长名称或描述。
5. standard_name：变量的标准名称，根据特定的科学标准命名。
6. valid_min：变量的有效最小值。
7. valid_max：变量的有效最大值。
8. missing_value：缺失值，用于标识数据中缺失或无效的值。
9. fill_value：填充值，用于标识数据中未填充的位置。
10. calendar：时间变量的日历类型，例如 "standard"、"gregorian" 等。
11. time_units：时间变量的单位，例如 "seconds since 1970-01-01"。
12. coordinates：变量所依赖的坐标变量。

# Xarray读取